In [1]:
from paratext import TrainingCorpus, TrainingText, Snippet

corpus = TrainingCorpus('../train')

In [2]:
text = TrainingText.from_file('../train/98-0.xml')

In [3]:
from boltons.iterutils import windowed

data = []
for path in corpus.paths():
    
    text = TrainingText.from_file(path)
    
    front = text.get_front()
    body = text.get_body_beginning()
    
    for lines in windowed(front.splitlines(), 10):
        snippet = Snippet('\n'.join(lines))
        data.append(snippet.features(dict(front=1)))
    
    for lines in windowed(body.splitlines(), 10):
        snippet = Snippet('\n'.join(lines))
        data.append(snippet.features(dict(front=0)))

In [4]:
len(data)

2697

In [5]:
import pandas as pd

df = pd.DataFrame(data)

In [6]:
f

,blank_line_ratio,caps_ratio,cd_ratio,chapter_ratio,digit_ratio,dt_ratio,front,jj_ratio,jjr_ratio,jjs_ratio,...,rb_ratio,rbr_ratio,rbs_ratio,vb_ratio,vbd_ratio,vbg_ratio,vbn_ratio,vbp_ratio,vbz_ratio,word_count
0,0.888889,0.073171,0.000000,0.000,0.000000,0.000000,1,0.250000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,4
1,0.888889,0.073171,0.000000,0.000,0.000000,0.000000,1,0.250000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,4
2,0.800000,0.333333,0.000000,0.000,0.000000,0.000000,1,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,7
3,0.777778,0.333333,0.000000,0.000,0.000000,0.000000,1,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,7
4,0.700000,0.310811,0.000000,0.000,0.000000,0.000000,1,0.100000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,10
5,0.666667,0.310811,0.000000,0.000,0.000000,0.000000,1,0.100000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,10
6,0.777778,0.476190,0.000000,0.000,0.000000,0.000000,1,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6
7,0.777778,0.476190,0.000000,0.000,0.000000,0.000000,1,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6
8,0.700000,0.411765,0.125000,0.125,0.019608,0.000000,1,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8
9,0.666667,0.411765,0.125000,0.125,0.019608,0.000000,1,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8


In [31]:
X = df.drop('front', axis=1)
y = df['front']

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Logistic regression

In [42]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [43]:
model.score(X_test, y_test)

0.97037037037037033

In [45]:
model.predict_proba(X_test)

array([[ 0.06848373,  0.93151627],
       [ 0.00974247,  0.99025753],
       [ 0.00616501,  0.99383499],
       ..., 
       [ 0.02251699,  0.97748301],
       [ 0.01629072,  0.98370928],
       [ 0.24118678,  0.75881322]])